In [1]:
from census import Census
import pandas as pd

Initial Setup

In [2]:

API_KEY = "PERSONAL API KEY"
c = Census(API_KEY)

# Census variables of interest
ACS_VARS = {
    'population': 'B01003_001E',
    'median_age': 'B01002_001E',
    'median_household_income' : 'B19013_001E',
    'num_households' : 'B11001_001E',
    'num_people_in_households' : 'B11002_001E',
    'total_workers' : 'B08301_001E',
    'public_transportation_to_work' : 'B08301_010E',
    'car_to_work' : 'B08301_002E',
    'walk_to_work' : 'B08301_019E',
    'population_poverty_eligible' : 'B17001_001E',
   'population_below_poverty' : 'B17001_002E'
}
########################################
########################################
########## CHANGE CITIES KEY ###########
########################################
########################################
# Manually store state/city FIPS for census use
cities = {
    'Arizona Diamondbacks': {'state': '04', 'place': '55000', 'team' : 'ARI'}, # Arizona Diamondbacks
    'Oakland Athletics': {'state': '06', 'place': '53000', 'team' : 'OAK'}, # Oakland Athletics
    'Atlanta Braves': {'state': '13', 'place': '04000', 'team' : 'ATL'}, # Atlanta Braves
    'Baltimore Orioles': {'state': '24', 'place': '04000', 'team' : 'BAL'}, # Baltimore Orioles
    'Boston Red Sox': {'state': '25', 'place': '07000', 'team' : 'BOS'}, # Boston Red Sox
    'Chicago White Sox': {'state': '17', 'place': '14000', 'team' : 'CHW'}, # Chicago White Sox
    'Chicago Cubs': {'state': '17', 'place': '14000', 'team' : 'CHC'}, # Chicago Cubs
    'Cincinnati Reds': {'state': '39', 'place': '15000', 'team' : 'CIN'}, # Cincinnati Reds
    'Cleveland Guardians': {'state': '39', 'place': '16000', 'team' : 'CLE'}, # Cleveland Guardians
    'Colorado Rockies': {'state': '08', 'place': '20000', 'team' : 'COL'}, # Colorado Rockies
    'Detroit Tigers': {'state': '26', 'place': '22000', 'team' : 'DET'}, # Detroit Tigers
    'Houston Astros': {'state': '48', 'place': '35000', 'team' : 'HOU'}, # Houston Astros
    'Kansas City Royals': {'state': '29', 'place': '38000', 'team' : 'KCR'}, # Kansas City Royals
    'Anaheim Angels': {'state': '06', 'place': '02000', 'team' : 'LAA'}, # Anaheim Angels
    'Los Angeles Dodgers': {'state': '06', 'place': '44000', 'team' : 'LAD'}, # Los Angeles Dodgers
    'Miami Marlins': {'state': '12', 'place': '45000', 'team' : 'MIA'}, # Miami Marlins
    'Milwaukee Brewers': {'state': '55', 'place': '53000', 'team' : 'MIL'}, # Milwaukee Brewers
    'Minnesota Twins': {'state': '27', 'place': '43000', 'team' : 'MIN'}, # Minnesota Twins
    'New York Yankees': {'state': '36', 'place': '51000', 'team' : 'NYY'}, # New York Yankees (Bronx)
    'New York Mets': {'state': '36', 'place': '51000', 'team' : 'NYM'}, # New York Mets (Queens)
    'Philadelphia Phillies': {'state': '42', 'place': '60000', 'team' : 'PHI'}, # Philadelphia Phillies
    'Pittsburgh Pirates': {'state': '42', 'place': '61000', 'team' : 'PIT'}, # Pittsburgh Pirates
    'San Diego Padres': {'state': '06', 'place': '66000', 'team' : 'SDP'}, # San Diego Padres
    'San Francisco Giants': {'state': '06', 'place': '67000', 'team' : 'SFG'}, # San Francisco Giants
    'Seattle Mariners': {'state': '53', 'place': '63000', 'team' : 'SEA'}, # Seattle Mariners
    'St. Louis Cardinals': {'state': '29', 'place': '65000', 'team' : 'STL'}, # St. Louis Cardinals
    'Tampa Bay Rays': {'state': '12', 'place': '63000', 'team' : 'TBR'}, # Tampa Bay Rays
    'Texas Rangers': {'state': '48', 'place': '04000', 'team' : 'TEX'}, # Texas Rangers
    'Washington Nationals': {'state': '11', 'place': '50000', 'team' : 'WSN'}, # Washington Nationals
    # Blue Jays Census data will not be gathered. Leaves us with 29 cities (Chicago teams and New York teams share a state/place combo)
    }



Access Census data

In [3]:
# Gather data
# Takes ~11 minutes
records = []
years = range(2012, 2020)

for city, fips in cities.items(): # For each city/state (29 total)
    for year in years: # For each year (2012-2019)
        c = Census(API_KEY, year=year) # Acquire census data for 27 cities for a given year
        try:
            data = c.acs1.get(
                list(ACS_VARS.values()),
                {'for': f'place:{fips["place"]}', 'in': f'state:{fips["state"]}'}
            )
            if data: # Save each city/year combo as its own observation
                result = data[0]
                result['year'] = year
                result['city'] = city
                result['team'] = fips['team']
                records.append(result)
        except Exception as e:
            print(f"Couldn't acquire data for {city} in {year}: {e}")

# Convert to DataFrame
df = pd.DataFrame(records)

# Rename columns
df = df.rename(columns={
    'B01003_001E': 'population',
    'B01002_001E': 'median_age',
    'B19013_001E' : 'median_household_income',
    'B11001_001E' : 'num_households',
    'B11002_001E' : 'num_people_in_households',
    'B08301_001E' : 'total_workers',
    'B08301_010E' : 'public_transportation_to_work',
    'B08301_002E' : 'car_to_work',
    'B08301_019E' : 'walk_to_work',
    'B17001_001E' : 'population_poverty_eligible',
    'B17001_002E' : 'population_below_poverty'
})

df['average_household_size'] = (df['num_people_in_households'] / df['num_households']).round(1)
# Commute to work types:
df['pct_public_transit'] = (df['public_transportation_to_work'] / df['total_workers'] * 100).round(1)
df['pct_car'] = (df['car_to_work'] / df['total_workers'] * 100).round(1)
df['pct_walk'] = (df['walk_to_work'] / df['total_workers'] * 100).round(1)
df['poverty_rate'] = (df['population_below_poverty'] / df['population_poverty_eligible'] * 100).round(1)

# Keep relevant columns
df = df[['team', 'city', 'year', 'population', 'median_age', 'median_household_income', 'average_household_size', 'pct_public_transit', 'pct_car', 'pct_walk', 'poverty_rate']]


Add Payroll Estimates (from stadium_data.csv) to Census data

In [5]:
stadium_df = pd.read_csv("stadium_data.csv")
stadium_df.columns = stadium_df.columns.str.lower()
df = df.merge(stadium_df[['team', 'year', 'payroll_est']], on=['team', 'year'], how='left')

In [6]:
df.head(15)

,team,city,year,population,median_age,median_household_income,average_household_size,pct_public_transit,pct_car,pct_walk,poverty_rate,payroll_est
0,ARI,Arizona Diamondbacks,2012,1488759.0,32.9,44153.0,2.8,3.2,87.3,1.8,24.1,67069833
1,ARI,Arizona Diamondbacks,2013,1513350.0,32.8,46601.0,2.9,4.0,86.8,1.8,23.6,80060500
2,ARI,Arizona Diamondbacks,2014,1537045.0,33.2,47929.0,2.9,3.4,86.1,1.9,23.3,89926500
3,ARI,Arizona Diamondbacks,2015,1563001.0,33.8,48452.0,2.9,3.5,87.1,1.8,22.3,64434000
4,ARI,Arizona Diamondbacks,2016,1615041.0,33.4,52062.0,2.9,3.0,87.3,1.6,20.3,78399500
5,ARI,Arizona Diamondbacks,2017,1626085.0,33.8,56696.0,2.9,2.8,87.4,1.6,16.8,106580200
6,ARI,Arizona Diamondbacks,2018,1660272.0,33.5,57957.0,2.9,3.0,87.0,1.4,15.6,134850600
7,ARI,Arizona Diamondbacks,2019,1680988.0,34.4,60931.0,2.8,2.9,86.7,1.5,15.6,124016266
8,OAK,Oakland Athletics,2012,400740.0,36.3,48196.0,2.5,19.2,66.1,4.3,22.0,61202500
9,OAK,Oakland Athletics,2013,406228.0,36.0,54394.0,2.6,18.4,65.7,4.1,19.5,69440000


In [7]:
df.to_csv("census_2012_2019.csv", index = False)